# 1. Importing Libraries & Dataset

In [74]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy.stats

In [ ]:
df = pd.read_csv("data/insurance_marketing_data.csv")

# 2. Data Overview

In [76]:
df.head()

,Client ID,Client Type,Number of Customers,Montly Target,Zip Code,Calendardate,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Number of Competition
0,ID-987275,Medium Facility,2800,125,1003,16-01-2014,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low
1,ID-987275,Medium Facility,2800,125,1003,16-02-2014,3409460,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322500.0,Low
2,ID-987275,Medium Facility,2800,125,1003,18-03-2014,10228384,75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low
3,ID-987275,Medium Facility,2800,125,1003,18-04-2014,17047304,123,0.0,0.0,0.0,0.0,3547500.0,1290000.0,0.0,0.0,Low
4,ID-987275,Medium Facility,2800,125,1003,19-05-2014,23866224,171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2976 entries, 0 to 2975
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Client ID              2976 non-null   object 
 1   Client Type            2976 non-null   object 
 2   Number of Customers    2976 non-null   int64  
 3   Montly Target          2976 non-null   int64  
 4   Zip Code               2976 non-null   int64  
 5   Calendardate           2976 non-null   object 
 6   Amount Collected       2976 non-null   int64  
 7   Unit Sold              2976 non-null   int64  
 8   Campaign (Email)       2976 non-null   float64
 9   Campaign (Flyer)       2976 non-null   float64
 10  Campaign (Phone)       2976 non-null   float64
 11  Sales Contact 1        2976 non-null   float64
 12  Sales Contact 2        2976 non-null   float64
 13  Sales Contact 3        2976 non-null   float64
 14  Sales Contact 4        2976 non-null   float64
 15  Sale

# 3.  Feature Additions and Engineering

In [78]:
df['Calendardate'] = pd.to_datetime(df['Calendardate'])

# 4. Simple EDA

In [79]:
# Distribution of Client Types
df['Client Type'].value_counts(normalize=True)

Large Facility      0.459677
Small Facility      0.282258
Medium Facility     0.169355
Private Facility    0.088710
Name: Client Type, dtype: float64

In [80]:
# Mean Amount Collected by Client Type
df.groupby('Client Type')[['Amount Collected']].mean().round(0)

,Amount Collected
Client Type,
Large Facility,19998805.0
Medium Facility,40759968.0
Private Facility,5030246.0
Small Facility,1637759.0


From the simple exploratory analysis, it seems that the most to least profitable Facility follows:
- Medium Facility
- Large Facility
- Private Facility
- Small Facility

In [81]:
pd.crosstab(df['Number of Competition'], df['Client Type'], margins=True, normalize='columns')

Client Type,Large Facility,Medium Facility,Private Facility,Small Facility,All
Number of Competition,,,,,
High,0.166667,0.166667,0.166667,0.166667,0.166667
Low,0.833333,0.833333,0.833333,0.833333,0.833333


In [82]:
df.groupby('Number of Competition').mean(numeric_only=True).round(0)

,Number of Customers,Montly Target,Zip Code,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5
Number of Competition,,,,,,,,,,,,,
High,1457.0,75.0,1003.0,29747888.0,213.0,105399.0,994047.0,45198.0,146946.0,2685333.0,1786754.0,72172.0,8453.0
Low,1457.0,75.0,1003.0,14455701.0,103.0,150862.0,623693.0,26693.0,128220.0,1890916.0,1883634.0,70482.0,15865.0


# 5. Statistical Analysis
## 5.1 Impact of (Marketing Strategies & Sales Contacts) on (Sales):

In [83]:
# Correlation between marketing strategies and Amount Collected
cm = sns.light_palette('green', as_cmap=True)
correlation_analysis = df[['Amount Collected','Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
       'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3',
       'Sales Contact 4', 'Sales Contact 5']].corr()['Amount Collected'].reset_index()
correlation_analysis.columns = ['Impacting Variable', 'Correlation Coefficient']
correlation_analysis = correlation_analysis[correlation_analysis['Impacting Variable'] != 'Amount Collected']
correlation_analysis = correlation_analysis.sort_values('Correlation Coefficient', ascending=False)
correlation_analysis.style.background_gradient(cmap=cm, subset=['Correlation Coefficient']).format("{:.2f}",subset=['Correlation Coefficient'])


,Impacting Variable,Correlation Coefficient
5,Sales Contact 2,0.55
2,Campaign (Flyer),0.44
6,Sales Contact 3,0.36
4,Sales Contact 1,0.28
1,Campaign (Email),0.25
7,Sales Contact 4,0.24
8,Sales Contact 5,0.10
3,Campaign (Phone),0.03


The correlation between which (marketing campagin strategies) & (amount_collected) from most to least successful:
- Flyers
- Email
- Phone

The correlation between which (Sales_Contact) & (Amount_Collected) from most to least successful:
- Contact 2
- Contact 3
- Contact 1
- Contact 4
- Contact 5

In [84]:
correlation_analysis = df.groupby('Client Type')[['Amount Collected','Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)', 'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3',
       'Sales Contact 4', 'Sales Contact 5']].corr()['Amount Collected'].reset_index()
correlation_analysis.columns = ['Client Type', 'Impacting Variable','Correlation Coefficient']
correlation_analysis = correlation_analysis[correlation_analysis['Impacting Variable'] != 'Amount Collected'].sort_values(['Client Type', 'Correlation Coefficient'], ascending=False)
cm = sns.light_palette('green', as_cmap=True)
correlation_analysis.style.background_gradient(cmap=cm, subset=['Correlation Coefficient']).format("{:.2f}", subset=['Correlation Coefficient'])


,Client Type,Impacting Variable,Correlation Coefficient
32,Small Facility,Sales Contact 2,0.22
33,Small Facility,Sales Contact 3,0.07
28,Small Facility,Campaign (Email),0.06
29,Small Facility,Campaign (Flyer),0.04
34,Small Facility,Sales Contact 4,0.02
35,Small Facility,Sales Contact 5,0.00
31,Small Facility,Sales Contact 1,-0.02
30,Small Facility,Campaign (Phone),nan
23,Private Facility,Sales Contact 2,0.57
20,Private Facility,Campaign (Flyer),0.28


Key-Takeaways:
- Sales_Contact_1: Overall has a negative impact in various facilities Sectors
- Sales_Contact_5: Under performing within the Smaller & Larger Facilities Sector
- Campaign_Email: Under performing within the Private Facilities Sector

# 6. Linear Regression Analysis

## 6.1 Linear Regression Analysis (Market Sales and Strategies)

In [85]:
# Modeling a Linear Regression Analysis on Sales on Marketing Strategies & Sales Contacts
import statsmodels.api as sm
import statsmodels.formula.api as smf
# Standardize column names for regression formula compatibility
df.columns = [mystring.replace(" ","_") for mystring in df.columns]
df.columns = [mystring.replace("(","") for mystring in df.columns]
df.columns = [mystring.replace(")","") for mystring in df.columns]
results = smf.ols('Amount_Collected ~ Campaign_Email+Campaign_Flyer+Campaign_Phone+Sales_Contact_1+Sales_Contact_2+Sales_Contact_3+Sales_Contact_4+Sales_Contact_5', data = df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       Amount_Collected   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.478
Method:                 Least Squares   F-statistic:                     342.1
Date:                Tue, 13 Jan 2026   Prob (F-statistic):               0.00
Time:                        01:15:46   Log-Likelihood:                -54512.
No. Observations:                2976   AIC:                         1.090e+05
Df Residuals:                    2967   BIC:                         1.091e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        1.481e+06   5.12e+05     

In [86]:
# Extracting significant coefficients where p-value is less than 0.05
coef_results = pd.read_html(results.summary().tables[1].as_html(), header = 0, index_col=0)[0].reset_index()
coef_results = coef_results[coef_results['P>|t|'] <= 0.05][['index', 'coef']]
coef_results.columns = ['Variable', 'Coefficient (Impact)']
coef_results = coef_results[coef_results['Variable'] != 'Intercept']
coef_results = coef_results.sort_values('Coefficient (Impact)', ascending=False)
print(coef_results)

          Variable  Coefficient (Impact)
7  Sales_Contact_4               10.9478
4  Sales_Contact_1                4.2368
5  Sales_Contact_2                3.6382
2   Campaign_Flyer                3.3376
6  Sales_Contact_3                2.3432


## 6.2 Linear Regression by Client_Types (Small, Medium, Large, Private)

In [87]:
# Modeling a Linear regression analysis By each Client Type (Small, Medium, Large, Private)

consolidated_summary = pd.DataFrame()

for acctype in df['Client_Type'].unique():
    temp_data = df[df['Client_Type'] == acctype]

    results = smf.ols(
        'Amount_Collected ~ Campaign_Email + Campaign_Flyer + Campaign_Phone + '
        'Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3 + '
        'Sales_Contact_4 + Sales_Contact_5',
        data=temp_data
    ).fit()

    coef_df = pd.read_html(
        results.summary().tables[1].as_html(),
        header=0,
        index_col=0
    )[0].reset_index()

    coef_df = coef_df[coef_df['P>|t|'] < 0.05][['index', 'coef']]
    coef_df.columns = ['Variable', 'Coefficient (Impact)']
    coef_df['Account Type'] = acctype
    coef_df = coef_df[coef_df['Variable'] != 'Intercept']
    coef_df = coef_df.sort_values('Coefficient (Impact)', ascending=False)
    print(acctype)
    print(coef_df)

    consolidated_summary = pd.concat(
        [consolidated_summary, coef_df],
        ignore_index=True
    )

Medium Facility
          Variable  Coefficient (Impact)     Account Type
2   Campaign_Flyer                4.1059  Medium Facility
5  Sales_Contact_2                3.5778  Medium Facility
4  Sales_Contact_1                3.1365  Medium Facility
6  Sales_Contact_3                2.1174  Medium Facility
Large Facility 
          Variable  Coefficient (Impact)     Account Type
4  Sales_Contact_1               11.6731  Large Facility 
7  Sales_Contact_4               10.6145  Large Facility 
5  Sales_Contact_2                4.0031  Large Facility 
2   Campaign_Flyer                2.7204  Large Facility 
6  Sales_Contact_3                2.0316  Large Facility 
3   Campaign_Phone               -3.5361  Large Facility 
Small Facility 
          Variable  Coefficient (Impact)     Account Type
5  Sales_Contact_2              0.810100  Small Facility 
3   Campaign_Phone              0.000001  Small Facility 
Private Facility
          Variable  Coefficient (Impact)      Account Type
5  Sal

In [88]:
consolidated_summary.sort_values('Coefficient (Impact)', ascending=False)

,Variable,Coefficient (Impact),Account Type
4,Sales_Contact_1,11.673100,Large Facility
5,Sales_Contact_4,10.614500,Large Facility
12,Sales_Contact_2,6.622300,Private Facility
0,Campaign_Flyer,4.105900,Medium Facility
6,Sales_Contact_2,4.003100,Large Facility
1,Sales_Contact_2,3.577800,Medium Facility
2,Sales_Contact_1,3.136500,Medium Facility
7,Campaign_Flyer,2.720400,Large Facility
3,Sales_Contact_3,2.117400,Medium Facility
8,Sales_Contact_3,2.031600,Large Facility


In [89]:
consolidated_summary.reset_index(drop = True,inplace=True)
consolidated_summary.columns = ['Variable','Return on Investment','Account Type'] # Regression coefficients are used as a proxy for ROI, not actual financial ROI
consolidated_summary['Return on Investment']= consolidated_summary['Return on Investment'].apply(lambda x: round(x,1))
consolidated_summary.style.background_gradient(cmap='RdYlGn')

,Variable,Return on Investment,Account Type
0,Campaign_Flyer,4.100000,Medium Facility
1,Sales_Contact_2,3.600000,Medium Facility
2,Sales_Contact_1,3.100000,Medium Facility
3,Sales_Contact_3,2.100000,Medium Facility
4,Sales_Contact_1,11.700000,Large Facility
5,Sales_Contact_4,10.600000,Large Facility
6,Sales_Contact_2,4.000000,Large Facility
7,Campaign_Flyer,2.700000,Large Facility
8,Sales_Contact_3,2.000000,Large Facility
9,Campaign_Phone,-3.500000,Large Facility


In [90]:
def format(x):
        return "${:.1f}".format(x)
consolidated_summary['Return on Investment']  = consolidated_summary['Return on Investment'].apply(format)

# 7. Conclusion:
- Using regression coefficients as a proxy for ROI, we estimate the revenue return generated per dollar spent across marketing channels and sales contacts.
The results show that marketing effectiveness varies significantly by account type, requiring tailored strategies rather than a one-size-fits-all approach.
- Case – Medium Facility:
Flyer campaigns deliver the highest ROI, generating approximately 4 dollars in revenue per dollar spent. Sales Contact 2 is the most effective, followed by Contacts 1 and 3, while other strategies show negligible impact and can be deprioritized to reduce costs.